<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Neural_Ordinary_Differential_Equations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install torchdiffeq

In [ ]:
from torchdiffeq import odeint
import torch
import torch.nn as nn

class ODEFunc(nn.Module):
    def __init__(self, hidden_dim):
        super(ODEFunc, self).__init__()
        self.linear = nn.Linear(hidden_dim, hidden_dim)
        self.activation = nn.ReLU()

    def forward(self, t, x):
        return self.activation(self.linear(x))

class NODE(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(NODE, self).__init__()
        self.fc_in = nn.Linear(input_dim, hidden_dim)
        self.odefunc = ODEFunc(hidden_dim)
        self.fc_out = nn.Linear(hidden_dim, output_dim)

    def forward(self, x, t):
        x = self.fc_in(x)
        x = odeint(self.odefunc, x, t)
        x = self.fc_out(x[-1])
        return x

# Example usage
input_dim = 2
hidden_dim = 16
output_dim = 1
t = torch.linspace(0, 1, 10)

model = NODE(input_dim, hidden_dim, output_dim)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(100):
    data = torch.randn(32, input_dim)
    target = torch.randn(32, output_dim)
    optimizer.zero_grad()
    output = model(data, t)
    loss = nn.MSELoss()(output, target)
    loss.backward()
    optimizer.step()
    print(f'Epoch {epoch + 1}, Loss: {loss.item():.4f}')